In [ ]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from scipy.ndimage import binary_erosion, sobel

class FRQIEncoder:
    def __init__(self, image_path):
        self.image_path = image_path
        self.qubits = 12
        self.qc = QuantumCircuit(self.qubits, self.qubits)
        self.backend_sim = AerSimulator()

    def image_normalization(self):
        image = Image.open(self.image_path).convert('LA')
        image = image.resize((32, 32))
        image_array = np.array(image)[:, :, 0] / 255.0
        return np.arcsin(image_array.flatten())

    def apply_hadamard(self):
        self.qc.h(range(2, self.qubits))

    def apply_frqi_encoding(self, pixel_data):
        for i, pixel in enumerate(pixel_data):
            if pixel != 0:
                self.c10mary(2 * pixel, format(i, '010b'), 0, 1, list(range(2, 12)))

    def c10mary(self, angle, bin_val, target, anc, controls):
        clist = [int(i) for i in bin_val]
        for i in range(len(clist)):
            if clist[i] == 0:
                self.qc.x(controls[-i-1])

        self.qc.mcry(angle, controls, target, anc)
        for i in range(len(clist)):
            if clist[i] == 0:
                self.qc.x(controls[-i-1])

    def apply_sobel_filter_3x3(self):
        # Aplicación de corrimientos en la ventana de 3x3 para obtener gradientes Gx y Gy
        shifts = [
            (1, 0),    # Y+ (desplazamiento hacia abajo)
            (1, 1),    # Y+X+ (diagonal hacia abajo-derecha)
            (0, 1),    # X+ (derecha)
            (-1, 1),   # Y-X+ (diagonal hacia arriba-derecha)
            (-1, 0),   # Y- (hacia arriba)
            (-1, -1),  # Y-X- (diagonal hacia arriba-izquierda)
            (0, -1),   # X- (izquierda)
            (1, -1),   # Y+X- (diagonal hacia abajo-izquierda)
            (0, 0)     # Central (sin desplazamiento)
        ]

        for idx, (y_shift, x_shift) in enumerate(shifts):
            # Aplicar rotaciones para simular el cambio de fase en función del corrimiento
            target_qubit = 2 + idx % (self.qubits - 2)  # Ciclar en qubits de posición
            if x_shift != 0:
                self.qc.cx(target_qubit, target_qubit + 1)
                self.qc.ry(np.pi / 128 * x_shift, target_qubit + 1)
                self.qc.cx(target_qubit, target_qubit + 1)
            if y_shift != 0:
                self.qc.cx(target_qubit + 1, target_qubit + 2)
                self.qc.rx(np.pi / 128 * y_shift, target_qubit + 2)
                self.qc.cx(target_qubit + 1, target_qubit + 2)

    def run_simulation(self):
        self.qc.measure(range(self.qubits), range(self.qubits))
        transpiled_circuit = transpile(self.qc, self.backend_sim)
        result = self.backend_sim.run(transpiled_circuit, shots=1000000).result()
        return result.get_counts(self.qc)

    def reconstruct_image(self, counts, pixel_data_length):
        genimg = np.zeros(pixel_data_length)
        for i in range(pixel_data_length):
            try:
                genimg[i] = np.sqrt(counts[format(i, '010b') + '01'] / 1000000)
            except KeyError:
                pass
        genimg = genimg * 255.0

        # Normalización y ajuste de contraste
        genimg = (genimg - genimg.min()) / (genimg.max() - genimg.min()) * 255

        # Binarización y erosión para mejorar bordes
        threshold = 120
        binarized_img = np.where(genimg > threshold, 1, 0)
        eroded_img = binary_erosion(binarized_img)

        contour_img = np.bitwise_xor(binarized_img, eroded_img)
        contour_img = contour_img.astype(int) * 255

        return contour_img

    def apply_classic_sobel(self, image):
        sx = sobel(image, axis=0, mode='constant')
        sy = sobel(image, axis=1, mode='constant')
        sobel_image = np.hypot(sx, sy)
        sobel_image = (sobel_image / sobel_image.max()) * 255
        return sobel_image

    def calculate_mse(self, original, reconstructed):
        return mean_squared_error(original.flatten(), reconstructed.flatten())

    def save_images(self, original, reconstructed):
        np.savetxt('imagen_original.txt', original.reshape(32, 32), fmt='%d')
        np.savetxt('imagen_generada.txt', reconstructed.reshape(32, 32), fmt='%d')

    def show_images(self, original, reconstructed, classic_sobel=None):
        fig, axs = plt.subplots(1, 3, figsize=(15, 5))

        axs[0].imshow(original.reshape(32, 32), cmap='gray', vmin=0, vmax=255)
        axs[0].set_title('Imagen Original')
        axs[0].axis('off')

        axs[1].imshow(reconstructed.reshape(32, 32), cmap='gray', vmin=0, vmax=255)
        axs[1].set_title('Reconstrucción Cuántica')
        axs[1].axis('off')

        if classic_sobel is not None:
            axs[2].imshow(classic_sobel.reshape(32, 32), cmap='gray', vmin=0, vmax=255)
            axs[2].set_title('Filtro Sobel Clásico')
            axs[2].axis('off')

        plt.savefig('Comparacion.png')
        plt.show()

    def draw_circuit_text(self, filename='circuito_32x32.txt'):
        with open(filename, 'w') as f:
            text_circuit = self.qc.draw(output='text')
            f.write(str(text_circuit))

    def run(self):
        pixel_data = self.image_normalization()
        self.apply_hadamard()
        self.apply_frqi_encoding(pixel_data)
        self.apply_sobel_filter_3x3()  # Aplicar el filtro Sobel cuántico con ventana 3x3
        counts = self.run_simulation()

        genimg = self.reconstruct_image(counts, len(pixel_data))
        original_img = np.sin(pixel_data) * 255.0

        classic_sobel = self.apply_classic_sobel(original_img.reshape(32, 32))
        mse = self.calculate_mse(original_img, genimg)
        print(f'Error cuadrático medio (MSE): {mse}')

        self.save_images(original_img, genimg)
        self.show_images(original_img, genimg, classic_sobel)
        self.draw_circuit_text()

# Ejecución del código con filtro Sobel cuántico y comparación clásica
if __name__ == '__main__':
    frqi_encoder = FRQIEncoder('img/lena.png')
    frqi_encoder.run()

# Desplazamientos 

from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from scipy.ndimage import binary_erosion, sobel

class FRQIEncoder:
    def __init__(self, image_path):
        self.image_path = image_path
        self.qubits = 12
        self.qc = QuantumCircuit(self.qubits, self.qubits)
        self.backend_sim = AerSimulator()

    def image_normalization(self):
        image = Image.open(self.image_path).convert('L')  # Convertir a escala de grises sin alfa
        image = image.resize((32, 32))
        image_array = np.array(image)  # Mantener valores originales en 0-255
        normalized_data = np.arcsin(image_array / 255.0)  # Normalizar solo para el cálculo cuántico
        return normalized_data.flatten(), image_array

    def apply_hadamard(self):
        self.qc.h(range(2, self.qubits))

    def apply_frqi_encoding(self, pixel_data):
        for i, pixel in enumerate(pixel_data):
            if pixel != 0:
                self.c10mary(2 * pixel, format(i, '010b'), 0, 1, list(range(2, 12)))

    def c10mary(self, angle, bin_val, target, anc, controls):
        clist = [int(i) for i in bin_val]
        for i in range(len(clist)):
            if clist[i] == 0:
                self.qc.x(controls[-i-1])

        self.qc.mcry(angle, controls, target, anc)
        for i in range(len(clist)):
            if clist[i] == 0:
                self.qc.x(controls[-i-1])

    def apply_displacements(self):
        #La lista shifts contiene nueve tuplas que representan los desplazamientos de un píxel central hacia cada una de las posiciones en una ventana de 3x3. Cada tupla tiene dos elementos:
        # El primer valor representa el desplazamiento en el eje Y (vertical).
        # El segundo valor representa el desplazamiento en el eje X (horizontal).
        # Esta lista de desplazamientos cubre todas las posiciones en una ventana de 3x3 alrededor de un píxel central. Aquí está el desglose de cada desplazamiento
        # Aplicación de corrimientos en la ventana de 3x3 para simular desplazamiento de píxeles
        shifts = [
            (1, 0),    # Y+ (desplazamiento hacia abajo)
            (1, 1),    # Y+X+ (diagonal hacia abajo-derecha)
            (0, 1),    # X+ (derecha)
            (-1, 1),   # Y-X+ (diagonal hacia arriba-derecha)
            (-1, 0),   # Y- (hacia arriba)
            (-1, -1),  # Y-X- (diagonal hacia arriba-izquierda)
            (0, -1),   # X- (izquierda)
            (1, -1),   # Y+X- (diagonal hacia abajo-izquierda)
            (0, 0)     # Central (sin desplazamiento)
        ]

        for idx, (y_shift, x_shift) in enumerate(shifts):
            target_qubit = 2 + idx % (self.qubits - 2)  # Ciclar en qubits de posición
            if x_shift != 0:
                self.qc.cx(target_qubit, target_qubit + 1)
                self.qc.ry(np.pi / 128 * x_shift, target_qubit + 1)
                self.qc.cx(target_qubit, target_qubit + 1)
            if y_shift != 0:
                self.qc.cx(target_qubit + 1, target_qubit + 2)
                self.qc.rx(np.pi / 128 * y_shift, target_qubit + 2)
                self.qc.cx(target_qubit + 1, target_qubit + 2)

    def run_simulation(self):
        self.qc.measure(range(self.qubits), range(self.qubits))
        transpiled_circuit = transpile(self.qc, self.backend_sim)
        result = self.backend_sim.run(transpiled_circuit, shots=1000000).result()
        return result.get_counts(self.qc)

    def reconstruct_image(self, counts, pixel_data_length):
        genimg = np.zeros(pixel_data_length)
        for i in range(pixel_data_length):
            try:
                genimg[i] = np.sqrt(counts[format(i, '010b') + '01'] / 1000000)
            except KeyError:
                pass
        genimg = genimg * 255.0

        # Normalización y ajuste de contraste
        genimg = (genimg - genimg.min()) / (genimg.max() - genimg.min()) * 255
        return genimg

    def show_image_displacements(self):
        # Normalizar y aplicar solo los desplazamientos
        pixel_data, original_img = self.image_normalization()
        self.apply_hadamard()
        self.apply_frqi_encoding(pixel_data)
        self.apply_displacements()  # Aplicar solo desplazamientos

        counts = self.run_simulation()
        displaced_img = self.reconstruct_image(counts, len(pixel_data))

        # Mostrar la imagen original y la imagen desplazada
        fig, axs = plt.subplots(1, 2, figsize=(10, 5))

        axs[0].imshow(original_img, cmap='gray', vmin=0, vmax=255)
        axs[0].set_title("Imagen Original")
        axs[0].axis('off')

        axs[1].imshow(displaced_img.reshape(32, 32), cmap='gray', vmin=0, vmax=255)
        axs[1].set_title("Imagen con Corrimientos Aplicados")
        axs[1].axis('off')

        plt.tight_layout()
        plt.show()

    def run(self):
        # Ejecución completa con filtro de Sobel cuántico
        pixel_data, _ = self.image_normalization()
        self.apply_hadamard()
        self.apply_frqi_encoding(pixel_data)
        self.apply_sobel_filter_3x3()  # Aplicar el filtro Sobel cuántico con ventana 3x3
        counts = self.run_simulation()

        genimg = self.reconstruct_image(counts, len(pixel_data))
        original_img = np.sin(pixel_data) * 255.0

        classic_sobel = self.apply_classic_sobel(original_img.reshape(32, 32))
        mse = self.calculate_mse(original_img, genimg)
        print(f'Error cuadrático medio (MSE): {mse}')

        self.save_images(original_img, genimg)
        self.show_images(original_img, genimg, classic_sobel)
        self.draw_circuit_text()

# Ejecución del código para ver solo los corrimientos
if __name__ == '__main__':
    frqi_encoder = FRQIEncoder('img/lena.png')
    frqi_encoder.show_image_displacements()  # Mostrar la imagen original y la imagen solo con corrimientos

In [ ]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from scipy.ndimage import binary_erosion, sobel

class FRQIEncoder:
    def __init__(self, image_path):
        self.image_path = image_path
        self.qubits = 12
        self.qc = QuantumCircuit(self.qubits, self.qubits)
        self.backend_sim = AerSimulator()

    def image_normalization(self):
        image = Image.open(self.image_path).convert('L')  # Convertir a escala de grises sin alfa
        image = image.resize((32, 32))
        image_array = np.array(image)  # Mantener valores originales en 0-255
        normalized_data = np.arcsin(image_array / 255.0)  # Normalizar solo para el cálculo cuántico
        return normalized_data.flatten(), image_array

    def apply_hadamard(self):
        self.qc.h(range(2, self.qubits))

    def apply_frqi_encoding(self, pixel_data):
        for i, pixel in enumerate(pixel_data):
            if pixel != 0:
                self.c10mary(2 * pixel, format(i, '010b'), 0, 1, list(range(2, 12)))

    def c10mary(self, angle, bin_val, target, anc, controls):
        clist = [int(i) for i in bin_val]
        for i in range(len(clist)):
            if clist[i] == 0:
                self.qc.x(controls[-i-1])

        self.qc.mcry(angle, controls, target, anc)
        for i in range(len(clist)):
            if clist[i] == 0:
                self.qc.x(controls[-i-1])

    def apply_displacements(self):
        # Desplazamiento uniforme hacia la derecha en toda la imagen
        x_shift = 0  # Desplazamiento en el eje X (derecha)
        y_shift = 1  # Sin desplazamiento en el eje Y

        # Aplicar el desplazamiento en todos los qubits de posición
        for idx in range(2, self.qubits - 2):  # Limitar a qubits de posición dentro del rango
            target_qubit = idx
            if x_shift != 0 and target_qubit + 1 < self.qubits:
                self.qc.cx(target_qubit, target_qubit + 1)
                self.qc.ry(np.pi / 64 * x_shift, target_qubit + 1)
                self.qc.cx(target_qubit, target_qubit + 1)
            if y_shift != 0 and target_qubit + 2 < self.qubits:
                self.qc.cx(target_qubit + 1, target_qubit + 2)
                self.qc.rx(np.pi / 64 * y_shift, target_qubit + 2)
                self.qc.cx(target_qubit + 1, target_qubit + 2)

    def run_simulation(self):
        self.qc.measure(range(self.qubits), range(self.qubits))
        transpiled_circuit = transpile(self.qc, self.backend_sim)
        result = self.backend_sim.run(transpiled_circuit, shots=1000000).result()
        return result.get_counts(self.qc)

    def reconstruct_image(self, counts, pixel_data_length):
        genimg = np.zeros(pixel_data_length)
        for i in range(pixel_data_length):
            try:
                genimg[i] = np.sqrt(counts[format(i, '010b') + '01'] / 1000000)
            except KeyError:
                pass
        genimg = genimg * 255.0

        # Normalización y ajuste de contraste
        genimg = (genimg - genimg.min()) / (genimg.max() - genimg.min()) * 255
        return genimg

    def show_image_displacements(self):
        # Normalizar y aplicar solo los desplazamientos
        pixel_data, original_img = self.image_normalization()
        self.apply_hadamard()
        self.apply_frqi_encoding(pixel_data)
        self.apply_displacements()  # Aplicar solo desplazamientos

        counts = self.run_simulation()
        displaced_img = self.reconstruct_image(counts, len(pixel_data))

        # Mostrar la imagen original y la imagen desplazada
        fig, axs = plt.subplots(1, 2, figsize=(10, 5))

        axs[0].imshow(original_img, cmap='gray', vmin=0, vmax=255)
        axs[0].set_title("Imagen Original")
        axs[0].axis('off')

        axs[1].imshow(displaced_img.reshape(32, 32), cmap='gray', vmin=0, vmax=255)
        axs[1].set_title("Imagen con Corrimientos Aplicados")
        axs[1].axis('off')

        plt.tight_layout()
        plt.show()

    def run(self):
        # Ejecución completa con filtro de Sobel cuántico
        pixel_data, _ = self.image_normalization()
        self.apply_hadamard()
        self.apply_frqi_encoding(pixel_data)
        self.apply_sobel_filter_3x3()  # Aplicar el filtro Sobel cuántico con ventana 3x3
        counts = self.run_simulation()

        genimg = self.reconstruct_image(counts, len(pixel_data))
        original_img = np.sin(pixel_data) * 255.0

        classic_sobel = self.apply_classic_sobel(original_img.reshape(32, 32))
        mse = self.calculate_mse(original_img, genimg)
        print(f'Error cuadrático medio (MSE): {mse}')

        self.save_images(original_img, genimg)
        self.show_images(original_img, genimg, classic_sobel)
        self.draw_circuit_text()

# Ejecución del código para ver solo los corrimientos
if __name__ == '__main__':
    frqi_encoder = FRQIEncoder('img/lena.png')
    frqi_encoder.show_image_displacements()  # Mostrar la imagen original y la imagen solo con corrimientos
